<a href="https://colab.research.google.com/github/garmartirosy/GithubActions/blob/main/ImportContributions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymrio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 16.9 MB/s eta 0:00:00
  Created wheel for pymrio: filename=pymrio-0.5.4-py3-none-any.whl size=140078 sha256=b504b714a1aec2c80c6f75787285489d54e9b9da1f9d10cd69190d7d45fa2ee2
  Stored in directory: /root/.cache/pip/wheels/f7/a2/98/de741b07dff4ebf8f79d17cd943f9d64f77c4fb2f1e08b7816
Successfully built pymrio
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.1.5
    Uninstalling openpyxl-3.1.5:
      Successfully uninstalled openpyxl-3.1.5


In [ ]:
import pymrio

pymrio.download_exiobase3(storage_folder='content', system='pxp', years=[2012])


exio_data = pymrio.parse_exiobase3(path="/content/content/IOT_2012_pxp.zip")


In [ ]:
import pandas as pd

# Step 1: Load region mapping CSV
region_mapping = pd.read_csv("/content/exio_country_names.csv")
region_mapping.columns = ["RegionCode", "Country"]

# Step 2: Load and transform the D_imp_reg data
import_contributions_df = exio_data.satellite.D_imp_reg

# Step 3: Reset index to bring stressor into a column
import_contributions_reset = import_contributions_df.reset_index()

# Step 4: Melt the DataFrame to long format
import_contributions_long = import_contributions_reset.melt(
    id_vars=["stressor"],            # Keep 'stressor' as a column
    var_name="Region",               # Regions become a single column
    value_name="ContributionAmount"  # Numerical values for contributions
)

# Step 5: Map regions to country names using the region mapping
import_contributions_long = pd.merge(
    import_contributions_long,
    region_mapping,
    how="left",
    left_on="Region",
    right_on="RegionCode"
)

# Step 6: Add a unique identifier (FlowUUID)
import_contributions_long["FlowUUID"] = import_contributions_long.apply(
    lambda row: f"{row['Region']}_{row['stressor']}", axis=1
)

# Step 7: Filter out rows with zero or null ContributionAmount
import_contributions_long = import_contributions_long[import_contributions_long["ContributionAmount"] != 0]

# Step 8: Drop unnecessary columns (optional)
import_contributions_long = import_contributions_long.drop(columns=["RegionCode"])

# Step 9: Save the final table to a CSV
import_contributions_long.to_csv("ImportContributionsTable.csv", index=False)

# Display a preview of the result
print("Final ImportContributions Table:")
print(import_contributions_long.head())

print("ImportContributions Table saved to 'ImportContributionsTable.csv'.")


Final ImportContributions Table:
                                            stressor Region  \
0  Taxes less subsidies on products purchased: Total     AT   
1                      Other net taxes on production     AT   
2  Compensation of employees; wages, salaries, & ...     AT   
3  Compensation of employees; wages, salaries, & ...     AT   
4  Compensation of employees; wages, salaries, & ...     AT   

   ContributionAmount  Country  \
0         6378.177477  Austria   
1         2004.911510  Austria   
2         2978.249700  Austria   
3        22845.624810  Austria   
4        17262.189151  Austria   

                                            FlowUUID  
0  AT_Taxes less subsidies on products purchased:...  
1                   AT_Other net taxes on production  
2  AT_Compensation of employees; wages, salaries,...  
3  AT_Compensation of employees; wages, salaries,...  
4  AT_Compensation of employees; wages, salaries,...  
ImportContributions Table saved to 'ImportContributio

In [ ]:
import_contributions_long

,stressor,Region,ContributionAmount,Country,FlowUUID
0,Taxes less subsidies on products purchased: Total,AT,6.378177e+03,Austria,AT_Taxes less subsidies on products purchased:...
1,Other net taxes on production,AT,2.004912e+03,Austria,AT_Other net taxes on production
2,"Compensation of employees; wages, salaries, & ...",AT,2.978250e+03,Austria,"AT_Compensation of employees; wages, salaries,..."
3,"Compensation of employees; wages, salaries, & ...",AT,2.284562e+04,Austria,"AT_Compensation of employees; wages, salaries,..."
4,"Compensation of employees; wages, salaries, & ...",AT,1.726219e+04,Austria,"AT_Compensation of employees; wages, salaries,..."
...,...,...,...,...,...
54532,Energy Carrier Net TMAR,WM,1.052491e+05,"Rest of World, Middle East",WM_Energy Carrier Net TMAR
54533,Energy Carrier Net TOTH,WM,4.704499e+04,"Rest of World, Middle East",WM_Energy Carrier Net TOTH
54534,Energy Carrier Net TRAI,WM,2.426882e+04,"Rest of World, Middle East",WM_Energy Carrier Net TRAI
54535,Energy Carrier Net TROA,WM,7.254860e+05,"Rest of World, Middle East",WM_Energy Carrier Net TROA


In [ ]:
exio_data.satellite.F_Y # Stressors/impacts of the final demand


region                                                                                      AT  \
category                                           Final consumption expenditure by households   
stressor                                                                                         
Taxes less subsidies on products purchased: Total                                          0.0   
Other net taxes on production                                                              0.0   
Compensation of employees; wages, salaries, & e...                                         0.0   
Compensation of employees; wages, salaries, & e...                                         0.0   
Compensation of employees; wages, salaries, & e...                                         0.0   
...                                                                                        ...   
Energy Carrier Net TMAR                                                                    0.0   
Energy Carrier Net TOTH                                                                    0.0   
Energy Carrier Net TRAI                                                                    0.0   
Energy Carrier Net TROA                                                               131692.0   
Energy Carrier Net LOSS                                                                    0.0   

region                                                                                                                                   \
category                                           Final consumption expenditure by non-profit organisations serving households (NPISH)   
stressor                                                                                                                                  
Taxes less subsidies on products purchased: Total                                                 0.0                                     
Other net taxes on production                                                                     0.0                                     
Compensation of employees; wages, salaries, & e...                                                0.0                                     
Compensation of employees; wages, salaries, & e...                                                0.0                                     
Compensation of employees; wages, salaries, & e...                                                0.0                                     
...                                                                                               ...                                     
Energy Carrier Net TMAR                                                                           0.0                                     
Energy Carrier Net TOTH                                                                           0.0                                     
Energy Carrier Net TRAI                                                                           0.0                                     
Energy Carrier Net TROA                                                                           0.0                                     
Energy Carrier Net LOSS                                                                           0.0                                     

region                                                                                          \
category                                           Final consumption expenditure by government   
stressor                                                                                         
Taxes less subsidies on products purchased: Total                                          0.0   
Other net taxes on production                                                              0.0   
Compensation of employees; wages, salaries, & e...                                         0.0   
Compensation of employees; wages, salaries, & e...                                         0

In [ ]:
exio_data.satellite.S # Direct stressor/impact coefficients


In [ ]:
exio_data.satellite.S_Y # Stressor/impact coefficients of the final demand


In [ ]:
exio_data.satellite.M # MRIO extension multipliers (total requirement factors of consumption)


region                                                     AT            \
sector                                             Paddy rice     Wheat   
stressor                                                                  
Taxes less subsidies on products purchased: Total           0  0.038572   
Other net taxes on production                               0 -0.029868   
Compensation of employees; wages, salaries, & e...          0  0.012633   
Compensation of employees; wages, salaries, & e...          0  0.114603   
Compensation of employees; wages, salaries, & e...          0  0.071593   
...                                                       ...       ...   
Energy Carrier Net TMAR                                     0  0.187353   
Energy Carrier Net TOTH                                     0  0.500155   
Energy Carrier Net TRAI                                     0  0.016087   
Energy Carrier Net TROA                                     0  4.572679   
Energy Carrier Net LOSS                                     0  1.297262   

region                                                                \
sector                                             Cereal grains nec   
stressor                                                               
Taxes less subsidies on products purchased: Total           0.031191   
Other net taxes on production                              -0.028553   
Compensation of employees; wages, salaries, & e...          0.010061   
Compensation of employees; wages, salaries, & e...          0.091947   
Compensation of employees; wages, salaries, & e...          0.056601   
...                                                              ...   
Energy Carrier Net TMAR                                     0.076280   
Energy Carrier Net TOTH                                     0.111207   
Energy Carrier Net TRAI                                     0.011314   
Energy Carrier Net TROA                                     0.433099   
Energy Carrier Net LOSS                                     0.980251   

region                                                                      \
sector                                             Vegetables, fruit, nuts   
stressor                                                                     
Taxes less subsidies on products purchased: Total                 0.022326   
Other net taxes on production                                    -0.023541   
Compensation of employees; wages, salaries, & e...                0.011485   
Compensation of employees; wages, salaries, & e...                0.100745   
Compensation of employees; wages, salaries, & e...                0.065578   
...                                                                    ...   
Energy Carrier Net TMAR                                           0.082434   
Energy Carrier Net TOTH                                           0.050655   
Energy Carrier Net TRAI                                           0.008410   
Energy Carrier Net TROA                                           0.673631   
Energy Carrier Net LOSS                                           0.770327   

region                                                        \
sector                                             Oil seeds   
stressor                                                       
Taxes less subsidies on products purchased: Total   0.020752   
Other net taxes on production                      -0.032973   
Compensation of employees; wages, salaries, & e...  0.007567   
Compensation of employees; wages, salaries, & e...  0.066963   
Compensation of employees; wages, salaries, & e...  0.042972   
...                                                      ...   
Energy Carrier Net TMAR                             0.126071   
Energy Carrier Net TOTH                             1.153899   
Energy Carrier Net TRAI                             0.013061   
Energy Carrier Net TROA                             0.682931   
Energy Carrier 

In [ ]:
exio_data.satellite.D_cba # Consumption based accounts per sector


In [ ]:
exio_data.satellite.D_pba # Production based accounts per sector


In [ ]:
exio_data.satellite.D_pba_reg # Production based accounts per region


In [ ]:
exio_data.satellite.D_cba_reg # Consumption based accounts per region

In [ ]:
exio_data.satellite.D_imp_reg # Import accounts per region


In [ ]:
exio_data.satellite.D_exp_reg # Export accounts per region


In [ ]:
Factor (includes FactorID and FlowUUID)
Industry (5-char)
IndustryFactor
Commodity (6-char)
CommodityFactor
ImportIndustryFactor
ImportCommodityFactor
ImportContributions
ImportMultiplierFactor


